In [ ]:
%store -r
import time
import os,json
import requests, operator
from dkube.sdk import *
from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeModelmonitor
from dkube.sdk.rsrcs.modelmonitor import DatasetClass,ModelType,DriftAlgo
from dkube.sdk.rsrcs.modelmonitor import DatasetFormat,DkubeModelmonitorAlert, TimeZone
from dkube.sdk.rsrcs.modelmonitor import DataType, ChannelOrder, ImageDataSavedFileFormat

In [ ]:
MONITOR_NAME = image_exp_config['MONITOR_NAME']
INPUT_TRAIN_TYPE = image_exp_config['INPUT_TRAIN_TYPE']
DKUBEUSERNAME = image_exp_config['DKUBEUSERNAME']
TOKEN = image_exp_config['TOKEN']
DKUBE_URL = image_exp_config['DKUBE_URL']
MODEL_NAME = image_exp_config['MODEL_NAME']
RUN_FREQUENCY = image_exp_config['RUN_FREQUENCY']
LIVE_DATASET = image_exp_config['LIVE_DATASET']
TRAINING_DATASET = image_exp_config['TRAINING_DATASET']

In [ ]:
def wait_for_deployment_running(deployment_id):
    status = None
    inference_url, inference = None, None
    while True:
        data = api.get_deployment(deployment_id)
        status = data.data.inferenceservice_deployment.parameters.generated.status.state
        inference = data.data.inferenceservice_deployment.parameters.inference
        inference_url = data.data.inferenceservice_deployment.parameters.generated.details.serving.servingurl
        if status == "RUNNING":
            break
        print("waiting for deployment to be running")
        time.sleep(api.wait_interval)
    return inference, inference_url

def get_dataset_version(username, dataset_name, version):
    dataset_versions = api.get_dataset_versions(username, dataset_name)
    versions = []
    for each_version in dataset_versions:
        if each_version["version"]["name"] == version:
            uuid = each_version["version"]["uuid"]
            return f"{version}:{uuid}"
        else:
            versions.append(each_version["version"]["name"])
    return f"dataset version {version} not found, available version are {versions}"

In [ ]:
api = DkubeApi(URL=os.getenv('DKUBE_URL'),token=os.getenv("DKUBE_USER_ACCESS_TOKEN"))

## Enabling logs

In [ ]:
while True:
    DEPLOYMENT_ID = api.get_deployment_id(name=MONITOR_NAME)
    if DEPLOYMENT_ID:
        break
    print("waiting for deployment to come up")
    time.sleep(api.wait_interval)
inference, INFERENCE_URL = wait_for_deployment_running(DEPLOYMENT_ID)
if not inference.enable_logs:
    print("Enabling logs")
    serving = DkubeServing(user=DKUBEUSERNAME, name=MONITOR_NAME)
    serving.update_enable_logs(enable_logs=True)
    api.update_inference(serving)
print("Inference is up at URL: ", INFERENCE_URL)
image_exp_config['INFERENCE_URL'] = INFERENCE_URL
image_exp_config['DEPLOYMENT_ID'] = DEPLOYMENT_ID
%store image_exp_config

## Initializing model monitor

In [ ]:
mm=DkubeModelmonitor(deployemnt_id = DEPLOYMENT_ID)

In [ ]:
mm.update_modelmonitor_basics(model_type=ModelType.Classification.value, 
                               input_data_type=DataType.Image.value,
                               data_timezone=TimeZone.UTC.value)

In [ ]:
mm.update_image_data_shape(height=200, width=200, channel=1)

In [ ]:
with open('thresholds.json') as f:
    thresholds = json.load(f)
mm.add_thresholds(thresholds=thresholds)

In [ ]:
training_data = f'{TRAINING_DATASET}:'+ DKUBEUSERNAME
train_data_version = get_dataset_version(DKUBEUSERNAME,
                                            TRAINING_DATASET, "v1")
labelled_data = LIVE_DATASET + ":"+ DKUBEUSERNAME
predict_data_format = "cloudeventlogs"

## Adding train, predict, and labelled datasets

In [ ]:
mm.add_datasources(data_class=DatasetClass.Train.value,name=training_data,
                   version=train_data_version,
                   s3_subpath=DEPLOYMENT_ID)

mm.add_datasources(data_class=DatasetClass.Predict.value,
                   data_format=predict_data_format)

mm.add_datasources(data_class=DatasetClass.Labelled.value,name=labelled_data,
                   data_format=DataType.Tabular.value, s3_subpath=DEPLOYMENT_ID + "/livedata",
                   predict_col="output",groundtruth_col="label",timestamp_col="timestamp")

## Health Monitoring

In [ ]:
mm.update_deployment_monitoring_details(enabled=True, frequency=1)

## Drift monitoring

In [ ]:
mm.update_drift_monitoring_details(enabled=True,
                                  frequency=RUN_FREQUENCY,
                                  image_train_data_savedfile_format="images_in_labelled_folder")

## Performance Monitoring

In [ ]:
mm.update_performance_monitoring_details(enabled=True,source_type="labelled_data",frequency=RUN_FREQUENCY)

## Creating model monitor

In [ ]:
id = api.modelmonitor_create(mm,wait_for_completion=True)

### Add alerts

#### Deployment Health Alert

In [ ]:
alert = DkubeModelmonitorAlert(name='latency_alert', alert_class = 'deployment_health')
alert.add_alert_condition(metric='latency_avg',threshold=300, op=operator.gt)
api.modelmonitor_add_alert(id,alert)

#### Data Drift Alert

In [ ]:
alert = DkubeModelmonitorAlert(name='image_drift_alert', alert_class = 'feature_drift')
alert.add_alert_condition(feature="image", threshold=0.2, op=operator.lt)
api.modelmonitor_add_alert(id,alert)

#### Performance Alert

In [ ]:
alert = DkubeModelmonitorAlert(name='accuracy_alert', alert_class = 'performance_decay')
alert.update_alert(metric='accuracy',threshold=0.9, op=operator.lt)
api.modelmonitor_add_alert(id,alert)

### Start the model monitor

In [ ]:
api.modelmonitor_start(DEPLOYMENT_ID)

### Cleanup

In [ ]:
CLEANUP = False
if CLEANUP:
    from time import sleep
    RETRIES = 4
    while RETRIES:
        mm = api.modelmonitor_get(DEPLOYMENT_ID)
        if mm["status"] and mm["status"]["state"].lower() != "active":
            break
        elif mm["status"] and mm["status"]["state"].lower() == "active":
            api.modelmonitor_stop(DEPLOYMENT_ID)
        RETRIES -= 1
        sleep(5)
    else:
        raise TimeoutError("modelmonitor failed to stopped")
    api.modelmonitor_delete(DEPLOYMENT_ID)